In [33]:
import pymysql
import pandas as pd

In [35]:
address_code = pd.read_excel('geocode_repeat.xlsx')

In [46]:
cursor.execute("select address_code_number,address_name,collected_latitude,collected_longitude,collector_id,manager_id,create_time,update_time from store.population_collector_address where address_code_number in "+str(tuple(address_code['address_code_number'])))
result =  cursor.fetchall()
village_with_latlon = pd.DataFrame(data = result)

In [48]:
from geopy import distance
def get_sl_distance(lon1,lat1,lon2,lat2):
    return distance.distance([lat1, lon1], [lat2, lon2]).km

In [49]:
def find_nearest_village(v_code,v_lat,v_lon,county):
    temp = county[county['address_code_number'] != v_code].copy()
    temp['sl_distance'] = temp.apply(lambda df: get_sl_distance(v_lon,v_lat,df['collected_longitude'],df['collected_latitude']),axis=1)
    target = temp[temp['sl_distance'] < 0.1]
    target['address_code_number'] = target['address_code_number'].astype(str)
    out = target['address_code_number'].values.tolist()
    if len(out) <1:
        return 1
    else:
        return ','.join(out)

In [50]:
village_with_latlon['repetition'] = village_with_latlon.apply(lambda df:find_nearest_village(df['address_code_number'],df['collected_latitude'],df['collected_longitude'],village_with_latlon),axis=1)

In [51]:
dt1 = village_with_latlon.loc[village_with_latlon['repetition'] != 1]

In [53]:
dt1.to_excel('./采集区域经纬度判断/重新采集repeat100m.xlsx')